# Calibrated Explanations for Binary Classification
## Stability and Robustness

Author: Tuwe Löfström (tuwe.lofstrom@ju.se)  
Copyright 2023 Tuwe Löfström  
License: BSD 3 clause
Sources:
1. ["Calibrated Explanations: with Uncertainty Information and Counterfactuals"](https://arxiv.org/abt/2305.02305) by [Helena Löfström](https://github.com/Moffran), [Tuwe Löfström](https://github.com/tuvelofstrom), Ulf Johansson, and Cecilia Sönströd.

### 1. Import packages

In [132]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [133]:
import pickle
import numpy as np

### 2 Import results from the pickled result file

In [134]:
with open('results_stab_rob.pkl', 'rb') as f:
    results = pickle.load(f)

### 3 Stability and Robustness
Create a table with the robustness and stability results. The stability results stem from experiements where the same model, calibration set and test set have been exaplained 30 times. The only source of variation is the random seed. The robustness restults stem from experiments where training and calibration sets have been randomly resampled before a new model have been trained and explained. The experiment where run 30 times and the test set was the same for all models. The robustness is measured in this way to avoid inferring perturbed instances which are not from the same distribution as the test instances being explained. The probability estimate of each of the models was computed on the same test set, as comparison to the robustness results. The expectationis that a stable and robust explanation method should result in low variance in the feature importance weights.

Everything was run on 25 datasets. See the `Classification_Experiment stab rob.py` for details on the experiment.

The tabulated results are the mean variance of the stability and robustness measured over the 30 runs and 20 instances. The variance is measured per instance and computed over the 30 runs on the feature importance weight of the most influential feature, defined as the feature most often having highest absolut feature importance weight. The average variance is computed over the 20 instances. The most influential feature is used since it is the feature that is most likely to be used in a decision but also the feature with the greatest expected variation (as a consequence of the weights having the highest absolute values). 

The results are printed as a latex table.

In [135]:
from scipy import stats as st
stab_rank = {}
stab_val = {}
stab_res = {}
rob_rank = {}
rob_val = {}
rob_proba = []
average_results = {}
for a in ['xGB', 'RF']:
    average_results[a+'_stab_ce'] = []
    average_results[a+'_stab_cce'] = []
    average_results[a+'_rob_ce'] = []
    average_results[a+'_rob_cce'] = []
    average_results[a+'_rob_proba'] = []

n = results['test_size']
r = results['num_rep']

print(' & xGB & xGB & xGB & xGB & xGB & RF & RF & RF & RF & RF \\\\')
print('Dataset & CE Stability & CCE Stability & CE Robustness & CCE Robustness & Model Variance & CE Stability & CCE Stability & CE Robustness & CCE Robustness & Model Variance \\\\\n\\hline')
for d in results:
    if d in ['test_size', 'num_rep']:
        continue
    print(d, end='')
    for a in results[d]:
        print(' & ', end='')
        stability = results[d][a]['stability']
        robustness = results[d][a]['robustness']
        
        for key in ['ce', 'cce']:    
            ranks = []
            values = []
            for j in range(n):
                rank = []
                value = []
                for i in range(r):
                    rank.append(np.argsort(np.abs(stability[key][i][j]['predict']))[-1:][0])
                ranks.append(rank)
                values.append(value)
            stab_rank[key] = st.mode(ranks, axis=1)[0] # Find most important feature per instance
            value = []
            for j in range(n):
                value.append([np.mean([stability[key][i][j]['predict'][stab_rank[key][j][0]] for i in range(r)]), np.var([stability[key][i][j]['predict'][stab_rank[key][j][0]] for i in range(r)])])
            stab_val[key] = value 
            
            ranks = []
            values = []
            for j in range(n):
                rank = []
                value = []
                for i in range(r):
                    rank.append(np.argsort(np.abs(robustness[key][i][j]['predict']))[-1:][0])
                ranks.append(rank)
                values.append(value)
            rob_rank[key] = st.mode(ranks, axis=1)[0] # Find most important feature per instance
            value = []
            for j in range(n):
                value.append([np.mean([robustness[key][i][j]['predict'][rob_rank[key][j][0]] for i in range(r)]), np.var([robustness[key][i][j]['predict'][rob_rank[key][j][0]] for i in range(r)])])
            rob_val[key] = value
        
        for inst in range(n):
            rob_proba.append(np.var([robustness['proba'][j][inst] for j in range(r)]))
        
        # print(stab_rank)
        # print(stab_val)
        average_results[a+'_stab_ce'].append(np.mean([t[1] for t in stab_val["ce"]]))
        average_results[a+'_stab_cce'].append(np.mean([t[1] for t in stab_val["cce"]]))
        average_results[a+'_rob_ce'].append(np.mean([t[1] for t in rob_val["ce"]]))
        average_results[a+'_rob_cce'].append(np.mean([t[1] for t in rob_val["cce"]]))
        average_results[a+'_rob_proba'].append(np.mean(rob_proba))
        # print(f'{np.mean([t[1] if t[1] > 1e-20 else 0 for t in stab_val["ce"]]):.1e} & {np.mean([t[1] if t[1] > 1e-20 else 0 for t in stab_val["cce"]]):.1e} & ',end='')
        # print(f'{np.mean([t[1] if t[1] > 1e-20 else 0 for t in rob_val["ce"]]):.1e} & {np.mean([t[1] if t[1] > 1e-20 else 0 for t in rob_val["cce"]]):.1e} & ',end='')
        print(f'{np.mean([t[1] for t in stab_val["ce"]]):.1e} & {np.mean([t[1] for t in stab_val["cce"]]):.1e} & ',end='')
        print(f'{np.mean([t[1] for t in rob_val["ce"]]):.1e} & {np.mean([t[1] for t in rob_val["cce"]]):.1e} & ',end='')
        print(f'{np.mean(rob_proba):.1e}', end='')
    print(' \\\\')
print('\\hline\nAverage', end='')
for a in ['xGB', 'RF']:
    print(' & ', end='')
    print(f'{np.mean(average_results[a+"_stab_ce"]):.1e} & {np.mean(average_results[a+"_stab_cce"]):.1e} & ',end='')
    print(f'{np.mean(average_results[a+"_rob_ce"]):.1e} & {np.mean(average_results[a+"_rob_cce"]):.1e} & ',end='')
    print(f'{np.mean(average_results[a+"_rob_proba"]):.1e}', end='')
print(' \\\\')

 & xGB & xGB & xGB & xGB & xGB & RF & RF & RF & RF & RF \\
Dataset & CE Stability & CCE Stability & CE Robustness & CCE Robustness & Model Variance & CE Stability & CCE Stability & CE Robustness & CCE Robustness & Model Variance \\
\hline
pc1req & 7.7e-35 & 7.7e-35 & 1.4e-02 & 1.4e-02 & 3.6e-02 & 1.8e-33 & 1.8e-33 & 1.4e-02 & 1.4e-02 & 2.4e-02 \\
haberman & 5.2e-34 & 5.2e-34 & 9.7e-03 & 9.7e-03 & 2.8e-02 & 1.2e-33 & 1.2e-33 & 1.1e-02 & 1.1e-02 & 2.4e-02 \\
hepati & 1.9e-34 & 1.9e-34 & 2.2e-02 & 2.2e-02 & 2.7e-02 & 5.2e-33 & 5.2e-33 & 1.4e-02 & 1.4e-02 & 2.3e-02 \\
transfusion & 2.8e-34 & 2.8e-34 & 8.7e-03 & 8.7e-03 & 2.5e-02 & 2.9e-33 & 2.9e-33 & 7.9e-03 & 7.9e-03 & 2.4e-02 \\
spect & 0.0e+00 & 0.0e+00 & 7.2e-03 & 7.2e-03 & 2.2e-02 & 1.5e-33 & 1.5e-33 & 6.0e-03 & 6.0e-03 & 2.0e-02 \\
heartS & 1.6e-33 & 1.6e-33 & 1.9e-02 & 1.9e-02 & 2.2e-02 & 4.3e-33 & 4.3e-33 & 1.7e-02 & 1.7e-02 & 2.0e-02 \\
heartH & 2.6e-33 & 2.6e-33 & 1.7e-02 & 1.7e-02 & 2.1e-02 & 4.9e-33 & 4.9e-33 & 1.1e-02 & 1.1e-0

As can be seen above, the stability is practically 0 for both factual CE (CE) and conterfactual CE (CCE), illustrating that the method is stable by definition. The robustness is also low, even if the mean variance is non-negligible. However, the robustness is low in comparison to the variance of the probability estimates used as reference, having on average about half the size of the variance of the probability estimates. This indicates that the method is fairly robust to perturbations such as variations of the calibration set and the model. 

### 4 Computing time
Now, lets look at the times taken to compute the explanations. The tabulated computing times are the average time in seconds. The results are printed as a latex table.

In [136]:
s_timer = []
r_timer = []
n = results['test_size']
r = results['num_rep']
average_time = {}
for a in ['xGB', 'RF']:
    average_time[a+'_stab_ce'] = []
    average_time[a+'_stab_cce'] = []
    average_time[a+'_rob_ce'] = []
    average_time[a+'_rob_cce'] = []

print(' & xGB & xGB & xGB & xGB & RF & RF & RF & RF \\\\')
print('Dataset & CE Stability & CCE Stability & CE Robustness & CCE Robustness  & CE Stability & CCE Stability& CE Robustness & CCE Robustness \\\\\n\\hline')
for d in results:
    if d in ['test_size', 'num_rep']:
        continue
    print(d, end='')
    for a in results[d]:
        print(' & ', end='')
        s_time = results[d][a]['stab_timer']
        r_time = results[d][a]['rob_timer']
        average_time[a+'_stab_ce'].append(np.mean([t/n for t in s_time["ce"]]))
        average_time[a+'_stab_cce'].append(np.mean([t/n for t in s_time["cce"]]))
        average_time[a+'_rob_ce'].append(np.mean([t/n for t in r_time["ce"]]))
        average_time[a+'_rob_cce'].append(np.mean([t/n for t in r_time["cce"]]))
        print(f'{np.mean([t/n for t in s_time["ce"]]):.2f} & {np.mean([t/n for t in s_time["cce"]]):.2f} & ',end='')
        print(f'{np.mean([t/n for t in r_time["ce"]]):.2f} & {np.mean([t/n for t in r_time["cce"]]):.2f}',end='')
    print(' \\\\')
print('\\hline\nAverage', end='')
for a in ['xGB', 'RF']:
    print(' & ', end='')
    print(f'{np.mean(average_time[a+"_stab_ce"]):.2f} & {np.mean(average_time[a+"_stab_cce"]):.2f} & ',end='')
    print(f'{np.mean(average_time[a+"_rob_ce"]):.2f} & {np.mean(average_time[a+"_rob_cce"]):.2f}',end='')
print(' \\\\')

 & xGB & xGB & xGB & xGB & RF & RF & RF & RF \\
Dataset & CE Stability & CCE Stability & CE Robustness & CCE Robustness  & CE Stability & CCE Stability& CE Robustness & CCE Robustness \\
\hline
pc1req & 0.09 & 0.09 & 0.09 & 0.09 & 0.23 & 0.23 & 0.23 & 0.23 \\
haberman & 0.06 & 0.06 & 0.06 & 0.06 & 0.14 & 0.14 & 0.14 & 0.15 \\
hepati & 0.19 & 0.23 & 0.20 & 0.23 & 0.47 & 0.55 & 0.46 & 0.55 \\
transfusion & 0.08 & 0.08 & 0.08 & 0.08 & 0.18 & 0.18 & 0.19 & 0.19 \\
spect & 0.15 & 0.15 & 0.15 & 0.15 & 0.34 & 0.34 & 0.33 & 0.32 \\
heartS & 0.16 & 0.19 & 0.16 & 0.19 & 0.37 & 0.44 & 0.38 & 0.45 \\
heartH & 0.17 & 0.20 & 0.17 & 0.20 & 0.42 & 0.48 & 0.44 & 0.52 \\
heartC & 0.20 & 0.23 & 0.21 & 0.25 & 0.51 & 0.56 & 0.50 & 0.58 \\
je4243 & 0.16 & 0.16 & 0.16 & 0.16 & 0.38 & 0.38 & 0.40 & 0.40 \\
vote & 0.11 & 0.11 & 0.11 & 0.10 & 0.25 & 0.25 & 0.24 & 0.24 \\
kc2 & 0.37 & 0.49 & 0.40 & 0.51 & 0.91 & 1.19 & 0.93 & 1.19 \\
\hline
Average & 0.16 & 0.18 & 0.16 & 0.18 & 0.38 & 0.43 & 0.39 & 0.44 \\


As can be seen, the computing time is fairly low for both CE and CCE. As expected, there is very little difference between the two methods. The computing time for the factual CE is slightly lower than for the CCE. This is because the CCE method computes the counterfactuals, which occasionally require some additional calculations. The main difference in time stems from the underlying model used, with xGBoost being about twice as fast as Random Forest. 